In [ ]:
# uncomment and run this cell if you have not installed dgl yet.
#!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

In [3]:
import os
import pickle
import sys
import numpy as np
import copy
import scipy
import scipy.io
import math

import torch
import torch.nn as nn  
from torch.utils.data import Dataset, DataLoader 

import dgl

from tqdm.notebook import tqdm
np.random.seed(123)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


## Dataset Class

In [4]:
class MolDataset(Dataset):
    def __init__(self, ids, filename = "feat.npy", targetfile = "qm7.mat", train = True):
        self.feats = np.load(filename, allow_pickle = True)
        self.feats = self.feats[ids] 
        self.trgs = scipy.io.loadmat(targetfile)['T']
        self.trgs = self.trgs[0, ids] 

        self.mean_feat = None 
        self.std_feat = None 

        self.mean_trg = None
        self.std_trg = None

        if train:
            self.mean_feat = np.concatenate(self.feats, axis = 0).mean()
            self.std_feat =  np.concatenate(self.feats, axis = 0).std()
            self.mean_trg = self.trgs.mean()
            self.std_trg = self.trgs.std()


    def __len__(self):
        return self.feats.shape[0]
    
    def __getitem__(self, idx):
        return self.feats[idx], self.trgs[idx], self.feats[idx].shape[0]

In [5]:
def collate(batch):
    #graphs = []
    feats = np.concatenate(list(map(lambda x: x[0], batch)),axis = 0)
    trgs = np.stack(list(map(lambda x: x[1], batch)), axis = 0)
    n_nodes =np.stack(list(map(lambda x: int(x[2]), batch)), axis = 0)
    feats = torch.tensor(feats).float()
    trgs = torch.tensor(trgs).float()
    return feats, trgs, n_nodes

def generate_batch_graph(n_nodes):
    graphs = [dgl.rand_graph(n_node, n_node + 1) for n_node in n_nodes]
    return dgl.batch(graphs)

In [6]:
def mae_fn(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def rmse_fn(y_true, y_pred):
    mse = np.square(np.subtract(y_true,y_pred)).mean() 
    return math.sqrt(mse)

## Model Class

In [7]:
class EnergyPredictor(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.embeddings = nn.Linear(in_dim, 128)
        self.mlp_1 = nn.Sequential(
            nn.Linear(128, 256), 
            nn.LeakyReLU(),
            nn.Linear(256, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 256)
        )
        self.mlp_2 = nn.Sequential(
            nn.Linear(256, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 1)
        )
        self.sum_pooling = dgl.nn.SumPooling()
    def forward(self, g, feat):
        feat = self.embeddings(feat)
        feat = self.mlp_1(feat)
        
        out =self.sum_pooling(g, feat)
        out = self.mlp_2(out)
        return out

## Functions

In [8]:
def train(model, loader, loss_fn, optimizer, mean, std, device = "cpu"):
    model.train()
    total_loss = []
    for idx, batch in tqdm(enumerate(loader), total = len(loader)):
        feats, trgs, n_nodes = batch
        batch_g = generate_batch_graph(n_nodes) 
        
        feats = (feats - mean["feat"]) / std["feat"] 

        feats = feats.to(device)
        trgs = trgs.to(device)
        batch_g = batch_g.to(device) 

        optimizer.zero_grad()
        out = model(batch_g, feats).squeeze()
        out = out * std["trg"] + mean["trg"]
        loss = loss_fn(out, trgs)
        loss.backward()
        optimizer.step() 
        total_loss.append(loss.detach().item())
    return np.array(total_loss).mean()

def eval(model, loader, metrics, mean, std, device = "cpu"):
    model.eval()
    batch_mae = []
    batch_rmse = []

    for idx, batch in tqdm(enumerate(loader), total = len(loader)):
        feats, trgs, n_nodes = batch
        batch_g = generate_batch_graph(n_nodes) 
        feats = feats.to(device)
        batch_g = batch_g.to(device) 

        feats = (feats - mean["feat"]) / std["feat"]
        with torch.no_grad():
            out = model(batch_g, feats).squeeze()
            out = out * std["trg"] + mean["trg"]
        
        trgs = trgs.numpy()
        out = out.cpu().detach().numpy()
        
        mae = metrics["mae"](out, trgs)
        rmse = metrics["rmse"](out, trgs) 

        batch_mae.append(mae)
        batch_rmse.append(rmse)
    return np.array(batch_mae).mean(), np.array(batch_rmse).mean()
    #print("MAE: ", np.array(batch_mae).mean())
    #print("RMSE: ", np.array(batch_rmse).mean())

In [9]:
if not os.path.exists('qm7.mat'):
    os.system('wget http://www.quantum-machine.org/data/qm7.mat')
dataset = scipy.io.loadmat('qm7.mat')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Training

In [ ]:
split = 4 # in [0,1,2,3,4]
P_train = dataset['P'][list(range(0, split))+list(range(split+1, 5))].flatten()
P_test = dataset['P'][split]

In [ ]:
train_dataset = MolDataset(P_train, filename = "feats/feat.npy", train = True)
test_dataset = MolDataset(P_test, filename = "feats/feat.npy", train = False)

In [ ]:
mean_feat, std_feat = train_dataset.mean_feat, train_dataset.std_feat
mean_trg, std_trg = train_dataset.mean_trg, train_dataset.std_trg

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 128, collate_fn = collate)
test_loader = DataLoader(test_dataset, batch_size = 128, collate_fn = collate)

In [ ]:
model = EnergyPredictor(23).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 5e-4)
loss_fn = nn.L1Loss() 

mean = {
    "feat" : mean_feat,
    "trg" : mean_trg
}
std = {
    "feat" : std_feat,
    "trg" : std_trg
}
metrics = {
    "mae" : mae_fn,
    "rmse": rmse_fn
}

In [ ]:
n_epochs = 100
best_mae = 100

In [ ]:
for e in range(n_epochs):
    print("Epoch: ", e)
    train_loss = train(model, train_loader, loss_fn, optimizer, mean, std, device)
    mae, rmse  = eval(model, test_loader,metrics, mean, std, device)

    print("Train Loss: ", train_loss)
    print("Val MAE: ", mae)
    print("Val RMSE: ", rmse)

    if best_mae > mae:
        best_mae = mae
        torch.save(
            model.state_dict(),
            f"model/model_{split}.pt",
        )
        print("--------Save Model--------")

    print()

## Testing

In [16]:
model = EnergyPredictor(23).to(device)
metrics = {
    "mae" : mae_fn,
    "rmse": rmse_fn
}

In [17]:
split_maes = []
split_rmses = []
for split in [0,1,2,3,4]:
    model.load_state_dict(torch.load(f"model/model_{split}.pt"))
    
    P_train = dataset['P'][list(range(0, split))+list(range(split+1, 5))].flatten()
    P_test = dataset['P'][split]    
    
    train_dataset = MolDataset(P_train, filename = "feats/feat.npy", train = True)
    test_dataset = MolDataset(P_test, filename = "feats/feat.npy", train = False)

    mean_feat, std_feat = train_dataset.mean_feat, train_dataset.std_feat
    mean_trg, std_trg = train_dataset.mean_trg, train_dataset.std_trg
    
    test_loader = DataLoader(test_dataset, batch_size = 128, collate_fn = collate, shuffle = False)
    mean = {
        "feat" : mean_feat,
        "trg" : mean_trg
    }
    std = {
        "feat" : std_feat,
        "trg" : std_trg
    }
    best_mae, best_rmse = eval(model, test_loader, metrics, mean, std, device)
    split_maes.append(best_mae)
    split_rmses.append(best_rmse) 

print("Split\t\tMAE\t\tRMSE")
for split in [0,1,2,3,4]:
    print("{}\t\t{:.2f}\t\t{:.2f}".format(split, split_maes[split], split_rmses[split]))

mae_scores = np.array(split_maes)
rmse_scores = np.array(split_rmses)

mean_mae = mae_scores.mean()
std_mae = mae_scores.std()

mean_rmse = rmse_scores.mean()
std_rmse = rmse_scores.std() 

print("Mean MAE: ", mean_mae)
print("Std MAE: ", std_mae)

print("Mean RMSE: ", mean_rmse) 
print("Std RMSE: ", std_rmse)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Split		MAE		RMSE
0		15.78		20.37
1		14.61		19.19
2		15.15		19.42
3		15.14		20.32
4		14.68		18.79
Mean MAE:  15.071821
Std MAE:  0.4201839
Mean RMSE:  19.617573507520138
Std RMSE:  0.6249408660101979
